# Setup

In [ ]:
import pandapower.networks as pnet

from scipy.linalg import inv

In [ ]:
import sys
sys.path.insert(1, '..')

from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import fro_error, map_error, max_error, rrms_error

# Network simulation
To start, we simulate Gaussian and independent fluctuations for power demands of loads, close to their referece values.

In [ ]:
net = pnet.case6ww()
samples = 100
load_cv = 0.02

In [ ]:
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, samples)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)

# OLS Identification

In [ ]:
y_ols = inv(voltage.conj().T @ voltage) @ voltage.conj().T @ current
print(f'Frobenius error: {fro_error(y_bus, y_ols)}')
print(f'Max error: {max_error(y_bus, y_ols)}')
print(f'RRMS error: {rrms_error(y_bus, y_ols)}')
print(f'MAP error: {map_error(y_bus, y_ols)}')